Example of LLM (& final project)

In [ ]:
## Make sure these are all installed and restart the runtime
## Also make sure you have a GPU runtime.
# pick a small one
!pip install accelerate &> /dev/null

In [ ]:
### I mounted google drive then made directories for the llm
##! mkdir /content/drive/MyDrive/llms
##! mdkir /content/drive/MyDrive/llms/MaziyarPanahiT
##! mdkir /content/drive/MyDrive/llms/MaziyarPanahi/Llama-3-8B-Instruct-v0.4

### Change into the llm root directory
%cd /content/drive/MyDrive/llms

model_path = "/content/drive/MyDrive/llms/MaziyarPanahi/Llama-3-8B-Instruct-v0.4"
###
### This downloads an entire llm commenting out since I already did it once
###
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="MaziyarPanahi/Llama-3-8B-Instruct-v0.4", local_dir = "MaziyarPanahi/Llama-3-8B-Instruct-v0.4/")

#*** for homework, download small model into root directory on Colab, don't need to do all these steps;
#**he put this here because he downloaded a big model and doesn't want to download it over and over ***

In [ ]:
### First we need to import the collection of tools that allow
### us to work with the llm. The transformers package has the basic
### utility of working with llms whereas torch reference pytorch,
### the nn framework put out originally by Meta.

##**this is for this specific model from huggingface website

from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import pipeline
import torch

### Next let's make the model
model = AutoModelForCausalLM.from_pretrained(
    ## First load the pretrained model, recall we set the path to the model earlier
    model_path,
    ##Below we set defaults of the library useage
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    # attn_implementation="flash_attention_2"
)

## The tokenizers takes characters/words/word sets and turns them
## into numbers
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

## The streamer passes text to and from the tokenizer
streamer = TextStreamer(tokenizer)

## The pipline now builds the io we'll use
my_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    streamer=streamer
)

In [ ]:
# Then you can use my_pipeline to generate text.
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = my_pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
)

print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
#searching 3b for a good size